In [13]:
import time, os, getpass
if "set_dummy" not in locals():
    os.chdir('..')
set_dummy = 'dummy'
import multiprocessing,threading,urllib,ipykernel,tempfile,concurrent.futures,matplotlib,warnings,zipfile # fix for fastai2.basics
from datasets import *
from architectures import *
from learning import *
from eval_metrics import *
from losses import *
from parser_ import get_args
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *

In [14]:
name = """
id:e2_arch:h7_ds:liberty_loss:tripletMargin_mpos:1.0_mneg:1.0_lr:3.0_maxsets:2000_sigmas-v:e011_thr:0.00016_WF:Hessian_PG:new_masks:AMOS-masks_tps:5000000_CamsB:5_fliprot_fewcams_ep:20_bs:3072_pos:2_dups:0
""".strip()
model = load_hardnet(name)
outname=name.split('_')[1].split(':')[1]
outname+='lib+colo+notre'
outname+='-fewcams-'
outname+='pca128'
outname+='lib'
print(outname)

Loading HardNet ...
arch: ['h7']
h7lib+colo+notre-fewcams-pca128lib


In [5]:
ds = torch.load('Datasets/6Brown/liberty.pt')
patches = ds['patches']
labels = ds['labels']

In [6]:
resize_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(32),
    transforms.ToTensor()
])

def chunks(lst, n): # Yield successive n-sized chunks from lst
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [7]:
inputs = [resize_transform(c) for c in patches]
inputs = torch.stack(inputs).cuda()

In [8]:
descs = []
for batch in chunks(inputs, 1024):
    descs += [model(batch).cpu().data.numpy()]
descs = np.concatenate(descs)

In [9]:
ncmps = 128
pca = PCA(n_components=ncmps)
pca.fit(descs)

PCA(copy=True, iterated_power='auto', n_components=128, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [10]:
w=torch.load('Models/{}/{}.pt'.format(name,name))
arch=w['model_arch']
w['model_arch']=arch+'pca'+str(ncmps)
w['pca']=pca
torch.save(w, outname+'.pt')
print(w['model_arch'])

h8E512pca128
